In [1]:
import pandas as pd
#import torch
#import torchvision
#from torch import nn
#import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt
import pronouncing
import markovify
import random
import os
import re
import keras
from keras.models import Sequential
from keras.layers import LSTM 
from keras.layers.core import Dense
%matplotlib inline

### Inspired by Paul Mooney from Kaggle.com
###### https://www.kaggle.com/paultimothymooney/poetry-generator-rnn-markov

The idea is to use Markov chains to predict the words and use RNN to predict the lines and maintain the rhythems


In [2]:
# import the dataset and display some info
artist_file = 'eminem.txt'
with open(artist_file) as f:
    print(f.read(100))

FileNotFoundError: [Errno 2] No such file or directory: 'eminem.txt'

In [ ]:
# plot the frequencies of words
def plotWordFrequency(input):
    f = open(artist_file,'r')
    words = [x for y in [l.split() for l in f.readlines()] for x in y] 
    
#     # remove punctuation from each word
#     import string
#     table = str.maketrans('', '', string.punctuation)
#     words = [w.translate(table) for w in raw_words]
    
#     for w in set(words):
#         print(w)
    
    data = sorted([(w, words.count(w)) for w in set(words)], key = lambda x:x[1], reverse=True)[:40] 
    most_words = [x[0] for x in data]
    times_used = [int(x[1]) for x in data]
    plt.figure(figsize=(20,10))
    plt.bar(x=sorted(most_words), height=times_used, color = 'grey', edgecolor = 'black',  width=.5)
    plt.xticks(rotation=45, fontsize=18)
    plt.yticks(rotation=0, fontsize=18)
    plt.xlabel('Most Common Words:', fontsize=18)
    plt.ylabel('Number of Occurences:', fontsize=18)
    plt.title('Most Commonly Used Words: %s' % (artist_file), fontsize=24)
    plt.show()

In [ ]:
plotWordFrequency(artist_file)

In [3]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(2, input_shape=(2, 2), return_sequences=True))#Returns the vector sequence of dimension 4, input_ shape=(timesteps, data_ dim)
	for i in range(depth):
		model.add(LSTM(4, return_sequences=True))#Returns the vector sequence of dimension 8
	model.add(LSTM(2, return_sequences=True))#Returns the vector sequence of dimension 2
	model.summary()
	model.compile(optimizer='rmsprop',
              loss='mse')
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

In [4]:
def markov(text_file):
    # Markov Chain
    read = open(text_file, "r", encoding='utf-8').read()
    text_model = markovify.NewlineText(read)
    return text_model

In [5]:
# Determine number of syllables in line
def syllables(line):
    count = 0
    for word in line.split(" "):#loop every word in each sentence
        vowels = 'aeiouy'
        word = word.lower().strip("!@#$%^&*()_+-={}[];:,.<>/?-")
#         word = word.lower().strip(".:;?!")
        if len(word) == 0:
            continue
        if word[0] in vowels:#If the word begins with a vowel, + 1
            count +=1
        for index in range(1,len(word)):
            if word[index] in vowels and word[index-1] not in vowels:#If the previous letter is a vowel and the next is not, + 1
                count +=1
        if word.endswith('e'):#If the last letter of the last word is 'e', -1
            count -= 1
        if word.endswith('le'):#If the end of the word is 'le', + 1
            count+=1
        if count == 0:#Prevent algorithm errors
            count +=1
    return count / maxsyllables

In [6]:
# Make index of words that rhyme with your word
def rhymeindex(lyrics):
	if str(artist) + ".rhymes" in os.listdir(".") and train_mode == False:#If the document has been created and is not in the training process, 
		print ("loading saved rhymes from " + str(artist) + ".rhymes")
		return open(str(artist) + ".rhymes", "r",encoding='utf-8').read().split("\n")#the document will be loading and returns a list of rhymes.
	else:
		rhyme_master_list = []
		print ("Building list of rhymes:")
		for i in lyrics:
			word = re.sub(r"\W+", '', i.split(" ")[-1]).lower()#word:The last word of every lyric
			rhymeslist = pronouncing.rhymes(word)#rhymeslist:Rhyme words corresponding to each word
			rhymeslistends = [] #rhymeslistends:The last two letters of each word, ex: teacher, daughterer->er
			for i in rhymeslist:
				rhymeslistends.append(i[-2:])
			try:#rhymescheme:Save the most frequent rhyme letter ex: er
				rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
			except Exception:
				rhymescheme = word[-2:]
			rhyme_master_list.append(rhymescheme)
		rhyme_master_list = list(set(rhyme_master_list))#rhyme_ master_ list: delete the repeated rhymes, leaving only one
		reverselist = [x[::-1] for x in rhyme_master_list]
		reverselist = sorted(reverselist)
		rhymelist = [x[::-1] for x in reverselist]#Sort the rhymes in order
		print("List of Sorted 2-Letter Rhyme Ends:")
		print(rhymelist)
		f = open(str(artist) + ".rhymes", "w", encoding='utf-8')#Write the rhymes in order
		f.write("\n".join(rhymelist))
		f.close()
		return rhymelist

In [7]:
# Make index of rhymes that you use
def rhyme(line, rhyme_list):
	word = re.sub(r"\W+", '', line.split(" ")[-1]).lower()
	rhymeslist = pronouncing.rhymes(word)
	rhymeslistends = []
	for i in rhymeslist:
		rhymeslistends.append(i[-2:])
	try:
		rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
	except Exception:
		rhymescheme = word[-2:]
	try:
		float_rhyme = rhyme_list.index(rhymescheme)
		float_rhyme = float_rhyme / float(len(rhyme_list))
		return float_rhyme
	except Exception:
		float_rhyme = None
		return float_rhyme

In [8]:
# Separate each line of the input txt
def split_lyrics_file(text_file):
	text = open(text_file, encoding='utf-8').read()
	text = text.split("\n")
	while "" in text:
		text.remove("")
	return text

In [9]:
# Generate lyrics
def generate_lyrics(text_model, text_file):
	bars = []
	last_words = []
	lyriclength = len(open(text_file,encoding='utf-8').read().split("\n"))
	count = 0
	markov_model = markov(text_file)
	
	while len(bars) < lyriclength / 9 and count < lyriclength * 2:
		bar = markov_model.make_sentence(max_overlap_ratio = .49, tries=100)
		if type(bar) != type(None) and syllables(bar) < 1:
			def get_last_word(bar):
				last_word = bar.split(" ")[-1]
				if last_word[-1] in "!.?,":
					last_word = last_word[:-1]
				return last_word
			last_word = get_last_word(bar)
			if bar not in bars and last_words.count(last_word) < 3:
				bars.append(bar)
				last_words.append(last_word)
				count += 1
	return bars

In [10]:
# Build dataset
def build_dataset(lines, rhyme_list):
	dataset = []
	line_list = []
	for line in lines:
		line_list = [line, syllables(line), rhyme(line, rhyme_list)]
		dataset.append(line_list)
	x_data = []
	y_data = []
	for i in range(len(dataset) - 3):
		line1 = dataset[i    ][1:]
		line2 = dataset[i + 1][1:]
		line3 = dataset[i + 2][1:]
		line4 = dataset[i + 3][1:]
		x = [line1[0], line1[1], line2[0], line2[1]]
		x = np.array(x)
		x = x.reshape(2,2)
		x_data.append(x)
		y = [line3[0], line3[1], line4[0], line4[1]]
		y = np.array(y)
		y = y.reshape(2,2)
		y_data.append(y)
	x_data = np.array(x_data)
	y_data = np.array(y_data)
	return x_data, y_data

In [11]:
# Compose verse
def compose_rap(lines, rhyme_list, lyrics_file, model):
    rap_vectors = []
    human_lyrics = split_lyrics_file(lyrics_file)
    initial_index = random.choice(range(len(human_lyrics) - 1))
    initial_lines = human_lyrics[initial_index:initial_index + 2]
    starting_input = []
    for line in initial_lines:
        starting_input.append([syllables(line), rhyme(line, rhyme_list)])
    starting_vectors = model.predict(np.array([starting_input]).flatten().reshape(1, 2, 2))
    rap_vectors.append(starting_vectors)
    for i in range(100):
        rap_vectors.append(model.predict(np.array([rap_vectors[-1]]).flatten().reshape(1, 2, 2)))
    return rap_vectors

def vectors_into_song(vectors, generated_lyrics, rhyme_list):
    print ("\n\n")	
    print ("Writing verse:")
    print ("\n\n")
    def last_word_compare(rap, line2):
        penalty = 0 
        for line1 in rap:
            word1 = line1.split(" ")[-1]
            word2 = line2.split(" ")[-1]
            while word1[-1] in "?!,. ":
                word1 = word1[:-1]
            while word2[-1] in "?!,. ":
                word2 = word2[:-1]
            if word1 == word2:
                penalty += 0.2
        return penalty
    def calculate_score(vector_half, syllables, rhyme, penalty):
        desired_syllables = vector_half[0]
        desired_rhyme = vector_half[1]
        desired_syllables = desired_syllables * maxsyllables
        desired_rhyme = desired_rhyme * len(rhyme_list)
        fds = float(desired_syllables)
        fs = float(syllables)
        fdr = float(desired_rhyme)
        fr = float(rhyme)
        score = 1.0 - abs(fds - fs) + abs(fdr - fr) - penalty
        return score
    dataset = []
    for line in generated_lyrics:
        line_list = [line, syllables(line), rhyme(line, rhyme_list)]
        dataset.append(line_list)
    rap = []
    vector_halves = []
    for vector in vectors:
        vector_halves.append(list(vector[0][0])) 
        vector_halves.append(list(vector[0][1]))
    for vector in vector_halves:
        scorelist = []
        for item in dataset:
            line = item[0]
            if len(rap) != 0:
                penalty = last_word_compare(rap, line)
            else:
                penalty = 0
#             print(item[2])
            total_score = calculate_score(vector, item[1], item[2], penalty)
            score_entry = [line, total_score]
            scorelist.append(score_entry)
        fixed_score_list = [0]
        for score in scorelist:
            fixed_score_list.append(float(score[1]))
        max_score = max(fixed_score_list)
        for item in scorelist:
            if item[1] == max_score:
                rap.append(item[0])
                print (str(item[0]))
                for i in dataset:
                    if item[0] == i[0]:
                        dataset.remove(i)
                        break
                break     
    return rap

In [12]:
# training function
def train(x_data, y_data, model):
	model.fit(np.array(x_data), np.array(y_data),
			  batch_size=2,
			  epochs=5,
			  verbose=1)
	model.save_weights(artist + ".rap")

Training and Predicting

In [13]:
def main(depth, train_mode):
	model = create_network(depth)
	text_model = markov(text_file)
	if train_mode == True:
		bars = split_lyrics_file(text_file)
	if train_mode == False:
		bars = generate_lyrics(text_model, text_file)
	rhyme_list = rhymeindex(bars)
	if train_mode == True:
		x_data, y_data = build_dataset(bars, rhyme_list)
		train(x_data, y_data, model)
	if train_mode == False:
		vectors = compose_rap(bars, rhyme_list, text_file, model)
		rap = vectors_into_song(vectors, bars, rhyme_list)
		f = open(rap_file, "w", encoding='utf-8')
		for bar in rap:
			f.write(bar)
			f.write("\n")

In [14]:
depth = 6
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 10
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 2, 2)              40        
_________________________________________________________________
lstm_1 (LSTM)                (None, 2, 4)              112       
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 4)              144       
_________________________________________________________________
lstm_3 (LSTM)                (None, 2, 4)              144       
_________________________________________________________________
lstm_4 (LSTM)                (None, 2, 4)              144       
_________________________________________________________________
lstm_5 (LSTM)                (None, 2, 4)              144       
_________________________________________________________________
lstm_6 (LSTM)                (None, 2, 4)              1

I just want to sit there in one spot
Throw me to want to fuck with me and thinks
But now its so fuckin depressed
Oh I love you boy I can adjust
Im sticking to the one who looked past
Im on a song and tell em apart
From a place to stay in your songs
I dont know feels like my mom Rabbit
I call the cops came through the storm
Man fuck that you can form
But I know I want to be different
Hes so mad Try to hold ya I wasnt
And in the same one that made them just faint
Cause it feels like my mom
Ill be the type of life and the crossbones
All I know its just images
Well be right back and I know its lies
And to the mic home with both ankles
The light that youre trying to save yours
And in the front to the bottle of meds
You dare me to have none of your kids
But its just the way to fuse it
But I was the one to the limit
Till we get to this day on this list the shit
You act like they got Slim
To just come in this dream
Yeah Im right back and wallow
Then Im have to be without a cause
Its the rage t

In [16]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(6, input_shape=(2, 2), return_sequences=True))#Returns the vector sequence of dimension 4, input_ shape=(timesteps, data_ dim)
	for i in range(depth):
		model.add(LSTM(10, return_sequences=True))#Returns the vector sequence of dimension 8
	model.add(LSTM(2, return_sequences=True))#Returns the vector sequence of dimension 2
	model.summary()
	model.compile(optimizer='rmsprop',
              loss='mse')
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

In [17]:
depth = 6
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 10
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 2, 6)              216       
_________________________________________________________________
lstm_17 (LSTM)               (None, 2, 10)             680       
_________________________________________________________________
lstm_18 (LSTM)               (None, 2, 10)             840       
_________________________________________________________________
lstm_19 (LSTM)               (None, 2, 10)             840       
_________________________________________________________________
lstm_20 (LSTM)               (None, 2, 10)             840       
_________________________________________________________________
lstm_21 (LSTM)               (None, 2, 10)             840       
_________________________________________________________________
lstm_22 (LSTM)               (None, 2, 10)            

And Imma be what you see me on your lips
50 told me to piss the world feed it beans
They just dont know its just the cats meoww ow
Cause that aint what Im seein
When I let you walk in
And youd think that youll win
And all I need a handout
And shes on top of my balls blew out
Why do we say and you slash your own but
Now I know thats a mountain of doubt
And they need a new plan
Thought I said that I can
Do I want to go to sleep
I never say goodbye to the boon docks
The moral to the back of my cheeks
Or shot up on me Im a masochist
Huh yup Am I afraid of the fittest
So lets go all the girls to take your heart
Kiss my dick in the fuck you and no Gs
That Ill still be the same exact beliefs
I had the same one that made them just faint
All I do want you to do that dont
Too good to be a drug addict traits
He walks in the back of my classes
Kings never die I look at the times
Almost gave up to be overseas
I said you was on my own hands
Its bout to have none of your kids
Was king of the way to w

In [18]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))#Returns the vector sequence of dimension 4, input_ shape=(timesteps, data_ dim)
	for i in range(depth):
		model.add(LSTM(8, return_sequences=True))#Returns the vector sequence of dimension 8
	model.add(LSTM(2, return_sequences=True))#Returns the vector sequence of dimension 2
	model.summary()
	model.compile(optimizer='rmsprop',
              loss='mse')
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

depth = 8
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 16
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 2, 4)              112       
_________________________________________________________________
lstm_33 (LSTM)               (None, 2, 8)              416       
_________________________________________________________________
lstm_34 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_35 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_36 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_37 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_38 (LSTM)               (None, 2, 8)             

Cause you got a weapon drawn
Rejoice every time you do but if the thought
And how real he is or there ever will be no Bozo
Youre trying to give a fuck yall all of your stereo
I learn to give you enough to slap box
I dont know When me and you said about you like to
Just know that the more that I can fin hear
And I just found out that I suffer
Ive got to say this but in my back pocket
Im not afraid Im not gonna be alright Its funny
Grab another one bit the fuckin studios at
You think I sold em all in a doubt undoubtedly
They got the underground but I know you like that
Im never too far I want you too
If I let you stop me now fore I get the chance to blow
But I need a place to stay in your songs
And put hands up and spit out and my highs
But she dont want you Marshall
Cause you never seen a ass like my shoe size
But I got You better bow down on me please
Get the fuck I look like a job this aint the chorus
Im on the table while hes all over the goal post
Slim Shady The way she was pregnant

In [19]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))#Returns the vector sequence of dimension 4, input_ shape=(timesteps, data_ dim)
	for i in range(depth):
		model.add(LSTM(8, return_sequences=True))#Returns the vector sequence of dimension 8
	model.add(LSTM(2, return_sequences=True))#Returns the vector sequence of dimension 2
	model.summary()
	model.compile(optimizer='rmsprop',
              loss='mse')
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

depth = 6
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 12
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_52 (LSTM)               (None, 2, 4)              112       
_________________________________________________________________
lstm_53 (LSTM)               (None, 2, 8)              416       
_________________________________________________________________
lstm_54 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_55 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_56 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_57 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_58 (LSTM)               (None, 2, 8)             

Cum I think I just rap on Protools
Only time I make me hurl
But Slim what if he is all on me and thinks
Skip over the same one that made them just faint
Please Marshall please let me see you with the government
And we can just walk up out this slump
Now I hope the weed that Ive done
We got you on a song and I cant be choosey
And you think Im bout to spit hard as shit
But thats all they say its best to raise it
Fuck slittin her throat I just cant admit
What he wrote down the Valium and lots of drugs
And threw dirt on me on your onion rings
I can be its just so fuckin wet and soft
Turns right around in this movie theatre please
They asking me to come down in flames
I told em all down with my rhymes
I aint even hear the cacklin
Ha bitch you need to get was bittersweet
We bout to feel like singin
I go back on you smilin
Lets cut to the rest of my skin
Too bad bitch last night so I could itll all be gray
Like chopping til Im on fire to yesterday
Guts over fear I have been in the stands
Ya I

In [21]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))#Returns the vector sequence of dimension 4, input_ shape=(timesteps, data_ dim)
	for i in range(depth):
		model.add(LSTM(8, return_sequences=True))#Returns the vector sequence of dimension 8
	model.add(LSTM(2, return_sequences=True))#Returns the vector sequence of dimension 2
	model.summary()
	model.compile(optimizer='rmsprop',
              loss=keras.losses.CategoricalCrossentropy())
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

depth = 6
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 12
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_76 (LSTM)               (None, 2, 4)              112       
_________________________________________________________________
lstm_77 (LSTM)               (None, 2, 8)              416       
_________________________________________________________________
lstm_78 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_79 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_80 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_81 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_82 (LSTM)               (None, 2, 8)             

Ill be the greatest gift you can sit me here
That we just play with his own daughter
Or if the thought of being a rapper
Its me my name was it was either
Give me my tank is on and too much chaos
Its like this shits about to fall apart
Wouldnt piss on the side of a lineup
This free worlds got you so fast when your lonely
And why I just hope it dont stop
Give out from the front door and wake up
Fack Im gonna make me hurl
I think I rap like this and say fagot
So finally I broke up with a belt
Now before you get this one
Prove that if I could be one of these streets quickly
Its too late to stop Im standing in the shit
Three to the bed and up under the bus then split
I guarantee Ill be right back on that shiddit
Can you dig it You know the tic tac songs
Were running right back and the deals and no Gs
Now youre in it for Satan
My 90s was like Fuck it I scoop deep
Then a shock when I said Im way too soft
And Im gone I was full of matches
And by the way Ill bring her with me just for today
Mig

In [22]:
def create_network(depth):
	model = Sequential()
	model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))#Returns the vector sequence of dimension 4, input_ shape=(timesteps, data_ dim)
	for i in range(depth):
		model.add(LSTM(8, return_sequences=True))#Returns the vector sequence of dimension 8
	model.add(LSTM(2, return_sequences=True))#Returns the vector sequence of dimension 2
	model.summary()
	model.compile(optimizer='adam',
              loss='mse')
	if artist + ".rap" in os.listdir(".") and train_mode == False:
		model.load_weights(str(artist + ".rap"))
		print("loading saved network: " + str(artist) + ".rap") 
	return model

depth = 8
artist = "artist"
rap_file = "temporary_poem.txt"

maxsyllables = 16
text_file = "eminem_removed.txt"
train_mode = True        
main(depth, train_mode)
train_mode = False
main(depth, train_mode)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_92 (LSTM)               (None, 2, 4)              112       
_________________________________________________________________
lstm_93 (LSTM)               (None, 2, 8)              416       
_________________________________________________________________
lstm_94 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_95 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_96 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_97 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_98 (LSTM)               (None, 2, 8)            

And I cant just walk up out this abyss
But in the days that I made a mess of my body
Tell him that I dont know what time it is
Tell the public that I got a nice day B Rabbit
My worst day on forward just let loose but cant
Try cid and get my balls like thats my pigment
Its like I want to fuck with fairies
I do want you Now should I say the same pace looking boy
As long as you left I had an interview with DJ Clue
Still tryna get on a ten
Used to have none of them got clothes
Who run it you do is be the man
Jealous when I have a fight with you
And you think I got or give up that easy No
Know what its kinda like oh oh oh oh oh oh
Just one of my life is a stove top
Something I wont say no but she broke her vows
To tell Shady I love you from leapin off the pot
So you dont hurt me and Im taking names
He wont have it happen to them wolves
Why am I supposed to be perfectly honest
Wasnt cause he asked to put you in your kids
I hope you can fit your damaged goods
Ha ha got you caught up between h